In [6]:
import numpy as np
import numpy.linalg as la
import pymatgen.core as mg
import json
import time
from pymatgen.core.structure import Structure

import importlib
import utils
import cgcnn.cgcnn.data
importlib.reload(utils)
importlib.reload(cgcnn.cgcnn.data)
from utils import GraphConstructor
from cgcnn.cgcnn.data import GaussianDistance
from pymatgen.optimization.neighbors import find_points_in_spheres
import pandas as pd
from pymatgen.ext.matproj import MPRester
import cython
import csv
import torch
from random import randint
MAPI = "RDq2I57WXBqDBeTtPz"
mpr = MPRester(MAPI)

In [ ]:
test = mpr.get_structure_by_material_id("mp-20470")

In [2]:
with open('./datasets/matbench_sample_100_mp_e_form.json', 'r') as f:
    df = pd.read_json(f.read(), orient='index')
df['structure'] = df['structure'].apply(Structure.from_dict)
targets = list(df['e_form'].values)
structures = list(df['structure'])
targets = np.array(targets)
mean = np.mean(targets)
std = np.std(targets)

In [ ]:
start = time.time()
gc = GraphConstructor()
dataset = []
idx = 0
for crystal in structures:
    dataset.append(gc.structure_to_graph(crystal, target=targets[idx], sid=idx))
    #dataset[-1].y = torch.FloatTensor(targets[idx])
    idx += 1
end = time.time()
print(end-start)
print(dataset[1])
print(dataset[1].__dict__.items())

In [3]:
import ocpmodels
from ocpmodels.trainers import RegressionTrainer
from ocpmodels.trainers import EnergyTrainer
from ocpmodels.datasets.lmdb_dataset import LmdbDataset
from ocpmodels.datasets.lmdb_dataset import SinglePointLmdbDataset

from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

In [4]:
'''dataset_in = {
    'train': dataset[:90],
    'val': dataset[90:],
    #'test': dataset[90:],
    'normalizer_info': {
        "normalize_labels": True,
        "target_mean": mean,
        "target_std": std,
    }
}'''
train_src = "./datasets/matbench_sample_1000_e_form.lmdb"
dataset = [
{'src': train_src, 'normalize_labels': True, 'target_mean': mean, 'target_std': std}, # train set 
{'src': train_src}, # val set (optional)
]
task = {
    'dataset': 'single_point_lmdb',
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['formation energy'],
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}
'''model = {
    'name': 'schnet',
    'hidden_channels': 256,
    'num_filters': 128,
    'num_interactions': 3,
    'num_gaussians': 100,
    'cutoff': 6.0,
    'use_pbc': True,
    'regress_forces': False,
    'otf_graph': True,
}'''
'''model = {
    'name': 'cgcnn',
    'atom_embedding_size': 128,
    'fc_feat_size': 256,
    'num_fc_layers': 4,
    'num_graph_conv_layers': 5,
    'num_gaussians': 100,
    'cutoff': 6.0,
    'regress_forces': False,
    'use_pbc': True,
}'''
'''model = {
    'name': 'painn',
    'hidden_channels': 256,
    'num_layers': 4,
    'num_rbf': 128,
    'cutoff': 6.0,
    'max_neighbors': 12,
    'scale_file': 'configs/painn/painn_nb6_scaling_factors.pt',
    'regress_forces': False,
    'use_pbc': True
}'''

model = {
    'name': 'dimenetplusplus',
    'hidden_channels': 256,
    'out_emb_channels': 192,
    'num_blocks': 3,
    'cutoff': 6.0,
    'num_radial': 6,
    'num_spherical': 7,
    'num_before_skip': 1,
    'num_after_skip': 2,
    'num_output_layers': 3,
    'regress_forces': False,
    'use_pbc': True,
    'otf_graph': True
}

optim = {
    'batch_size': 32,
    'eval_batch_size': 32,
    'load_balancing': 'atoms',
    'num_workers': 0,
    'optimizer': 'AdamW',
    'optimizer_params': {"amsgrad": True},
    'lr_initial': .0001,
    'lr_gamma': .1,
    'lr_milestones': [1562,2343,3125],
    'warmup_steps': 468,
    'warmup_factor': 0.2,
    'max_epochs': 5,
    'scheduler': 'ReduceLROnPlateau',
    'mode': 'min',
    'factor': 0.8,
    'patience': 3,
    'energy_coefficient': 1.0,
    'ema_decay': 0.999,
    'clip_grad_norm': 10,
    'weight_decay': 0
}


In [5]:
trainer = RegressionTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optim,
    identifier='dime-test',
    run_dir="./",
    is_debug=True,
    cpu=True,
    print_every=1,
    seed=0,
    logger="tensorboard",
    local_rank=0,
    amp=False,
    from_lmdb=True,
)

True
{'src': './datasets/matbench_sample_1000_e_form.lmdb', 'normalize_labels': True, 'target_mean': -1.326414973042, 'target_std': 1.0543454730166244}
amp: false
cmd:
  checkpoint_dir: ./checkpoints\2022-06-16-10-34-24-dime-test
  commit: 506d431
  identifier: dime-test
  logs_dir: ./logs\tensorboard\2022-06-16-10-34-24-dime-test
  print_every: 1
  results_dir: ./results\2022-06-16-10-34-24-dime-test
  seed: 0
  timestamp_id: 2022-06-16-10-34-24-dime-test
dataset:
  normalize_labels: true
  src: ./datasets/matbench_sample_1000_e_form.lmdb
  target_mean: !!python/object/apply:numpy.core.multiarray.scalar
  - &id001 !!python/object/apply:numpy.dtype
    args:
    - f8
    - false
    - true
    state: !!python/tuple
    - 3
    - <
    - null
    - null
    - null
    - -1
    - -1
    - 0
  - !!binary |
    5j0i6P449b8=
  target_std: !!python/object/apply:numpy.core.multiarray.scalar
  - *id001
  - !!binary |
    P6zUW5ne8D8=
gpus: 0
logger: tensorboard
model: dimenetplusplus
model_att

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [ ]:
train_src = "./datasets/data.lmdb"
task = {
    'dataset': 'single_point_lmdb',
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}
model = {
    'name': 'painn',
    'hidden_channels': 1024,
    'num_layers': 3,
    'num_rbf': 128,
    'cutoff': 6.0,
    'max_neighbors': 50,
    'regress_forces': False,
    'use_pbc': True,
    'otf_graph': True
}
optim = {
    'batch_size': 8,
    'eval_batch_size': 8,
    'load_balancing': 'atoms',
    'num_workers': 0,
    'optimizer': 'AdamW',
    'optimizer_params': {"amsgrad": True},
    'lr_initial': 5.e-3,
    'scheduler': 'ReduceLROnPlateau',
    'mode': 'min',
    'factor': 0.8,
    'patience': 3,
    'max_epochs': 5,
    'energy_coefficient': 1,
    'ema_decay': 0.999,
    'clip_grad_norm': 10,
    'weight_decay': 0.0
}
dataset = [
{'src': train_src, 'normalize_labels': True, 'target_mean': -1.525913, 'target_std': 2.279365}, # train set 
{'src': train_src}, # val set (optional)
{'src': train_src} # test set (optional - writes predictions to disk)
]

trainer = EnergyTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optim,
    identifier="painn-plz",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    print_every=5,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)


In [ ]:
trainer.train()

In [ ]:
dataset = LmdbDataset({"src": "./datasets/data.lmdb"})
print(len(dataset))
print(dataset[0])

In [ ]:
import lmdb
import pickle
from torch_geometric.data import Data
db = lmdb.open(
    "./datasets/matbench_sample_1000_e_form.lmdb",
    map_size=1073741824 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)

In [ ]:


for i in range(len(dataset)):
    txn = db.begin(write=True)
    txn.put(f"{i}".encode("ascii"), pickle.dumps(dataset[i], protocol=-1))
    txn.commit()
    db.sync()
db.close()

In [ ]:
dataset = LmdbDataset({"src": "./datasets/matbench_sample_1000_e_form.lmdb"})
len(dataset)
print(len(dataset))
print(dataset[0])

In [ ]:
test = Data(pos=torch.LongTensor([5, 5]))
print(test.__dict__.items())
a = {k: v for k, v in test.__dict__.items() if v is not None}
print(a)
a_data = Data(**a)
print(a_data)
for k, v in test.__dict__.items():
    print(k, " WHA ", v)
    if v is not None:
        print("YA")